This notebook will be collected automatically at **6pm on Monday** from `/home/data_scientist/assignments/Week8` directory on the course JupyterHub server. If you work on this assignment on the course Jupyterhub server, just make sure that you save your work and instructors will pull your notebooks automatically after the deadline. If you work on this assignment locally, the only way to submit assignments is via Jupyterhub, and you have to place the notebook file in the correct directory with the correct file name before the deadline.

1. Make sure everything runs as expected. First, restart the kernel (in the menubar, select `Kernel` → `Restart`) and then run all cells (in the menubar, select `Cell` → `Run All`).
2. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed by the autograder.
3. Do not change the file path or the file name of this notebook.
4. Make sure that you save your work (in the menubar, select `File` → `Save and CheckPoint`)

# Problem 8.2. Social Media: Twitter

In this problem, we will use the twitter API to extract a set of tweets, and perform a sentiment analysis on twitter data to classify tweets as positive or negative.

In [ ]:
import numpy as np
import nltk
import tweepy as tw

from sklearn.utils import check_random_state
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from nose.tools import assert_equal, assert_is_instance, assert_true, assert_almost_equal
from numpy.testing import assert_array_equal

## Twitter Application

- Create a Twitter Application, and save the credentials into a file at `/home/data_scientist/Week8/notebooks/twitter.cred`. 

`twitter.cred` must have the following four credentials in order on separate lines:

```
Access Token
Access Token Secret
Consumer Key
Consumer Secret
```

Once you have stored your credientials, run the following code cells (you don't have to write any code in this section) and check if you are able to use the API.

In [ ]:
def connect_twitter_api(cred_file):
    
    # Order: Access Token, Access Token Secret, Consumer Key, Consumer SecretAccess
    with open(cred_file) as fin:
        tokens = [line.rstrip('\n') for line in fin if not line.startswith('#')]

    auth = tw.OAuthHandler(tokens[2], tokens[3])
    auth.set_access_token(tokens[0], tokens[1])

    return tw.API(auth)

In [ ]:
# Do NOT change file path or name of twitter.cred
api = connect_twitter_api('/home/data_scientist/Week8/notebooks/twitter.cred')
assert_equal(api.get_user('katyperry').screen_name, 'katyperry')
assert_equal(api.get_user('justinbieber').created_at.strftime('%Y %m %d %H %M %S'), '2009 03 28 16 41 22')
assert_equal(api.get_user('BarackObama').name, 'Barack Obama')

## Positive and negative tweets

We will first train a model on the NLTK twitter corpus, and use it to classify a set of tweets fetched from the Twitter API.

In [ ]:
from nltk.corpus import twitter_samples as tws

- Write a function that creates a training set from the NLTK twitter corpus. Positive tweets are in `positive_tweets.json`, while negative tweets are in `negative_tweets.json`. The `data` and `targets` ararys should be one-dimensional numpy arrays, where the first half are the positive tweets and the second half are the negative tweets. Every positive tweets should be assigned a numerical label of 1 in `targets`, and negative tweets 0.

In [ ]:
def get_pos_neg_tweets(corpus):
    '''
    Creates a training set from twitter_samples corpus.
    
    Parameters
    ----------
    corpus: The nltk.corpus.twitter_samples corpus.
    
    Returns
    -------
    A tuple of (data, targets)
    
    '''
    
    # YOUR CODE HERE
    
    return data, targets

In [ ]:
data, targets = get_pos_neg_tweets(tws)

In [ ]:
assert_is_instance(data, np.ndarray)
assert_is_instance(targets, np.ndarray)
assert_equal(len(data), 10000)
assert_equal(len(targets), 10000)
assert_array_equal(
    data[:5],
    [
        '#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top '
            'engaged members in my community this week :)',
        '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234'
            ' and we will be able to assist you :) Many thanks!',
        '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing '
            'track. When are you in Scotland?!',
        '@97sides CONGRATS :)',
        'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark '
            'on my fb profile :) in 15 days'
        ]
    )
assert_array_equal(
    data[5000:5005],
    [
        'hopeless for tmr :(',
        "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in "
            "2 months :(",
        '@Hegelbon That heart sliding into the waste basket. :(',
        '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
        'Dang starting next week I have "work" :('
        ]
    )
assert_array_equal(targets[:5000], [1] * 5000)
assert_array_equal(targets[5000:], [0] * 5000)

## Training

We train on 80% of the data, and test the performance on the remaining 20%.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.2, random_state=check_random_state(0)
    )

- Use unigrams, bigrams, and trigrams.
- Build a pipeline by using TfidfVectorizer and LinearSVC,
- Name the first step tf and the second step svc,
- Use default parameters for both TfidfVectorizer and LinearSVC, and
- Use English stop words,

In [ ]:
def train(X_train, y_train, X_test, random_state):
    '''
    Creates a document term matrix and uses SVM classifier to make document classifications.
    
    Parameters
    ----------
    X_train: A list of strings.
    y_train: A list of strings.
    X_test: A list of strings.
    random_state: A np.random.RandomState instance.
    
    Returns
    -------
    A tuple of (clf, y_pred)
    clf: A Pipeline instance.
    y_pred: A numpy array.
    '''
    
    # YOUR CODE HERE
    
    return clf, y_pred

In [ ]:
clf, y_pred = train(X_train, y_train, X_test, random_state=check_random_state(0))
score = accuracy_score(y_pred, y_test)
print("SVC prediction accuracy = {0:5.1f}%".format(100.0 * score))

In [ ]:
assert_is_instance(clf, Pipeline)
assert_is_instance(y_pred, np.ndarray)
tf = clf.named_steps['tf']
assert_is_instance(tf, TfidfVectorizer)
assert_is_instance(clf.named_steps['svc'], LinearSVC)
assert_equal(tf.stop_words, 'english')
assert_equal(tf.ngram_range, (1, 3))
assert_equal(len(y_pred), len(y_test))
assert_array_equal(y_pred[:10], [0, 1, 1, 0, 1, 0, 0, 0, 1, 1])
assert_array_equal(y_pred[-10:], [0, 0, 1, 1, 1, 0, 0, 1, 1, 0])
assert_almost_equal(score, 0.76400000)

## User timeline

- Use Tweepy's [user_timeline()](http://docs.tweepy.org/en/latest/api.html#API.user_timeline) to extract 20 tweets from a specified user. Specify the `max_id` parameter for reproducibility.

In [ ]:
def get_timeline(user, max_id):
    '''
    Fetches 20 tweets from "user".
    
    Parameters
    ----------
    user: A string. The ID or screen name of a Twitter user.
    max_id: An int. Returns only statuses with an ID less than
            (that is, older than) or equal to the specified ID.
    
    Returns
    -------
    A list of integers.
    '''
    
    # YOUR CODE HERE
    
    return timeline

In [ ]:
timeline1 = get_timeline('HillaryClinton', max_id=705395196817702912)

In [ ]:
timeline2 = get_timeline('realDonaldTrump', max_id=705362027901009921)

In [ ]:
assert_equal(
    [t.id for t in timeline1],
    [705395196817702912, 705176659457085440, 705176341877014528, 705160472727830528,
     705143770640044032, 705131896305541120, 705129859941584896, 705099211285667840,
     705075594451505153, 705061640031567872, 705029404641009664, 704887859514318848,
     704881748207861760, 704878458619498496, 704871605265047552, 704862926671249408,
     704856851721146369, 704851986966036480, 704850687641985024, 704849583160082434]
    )
assert_equal(
    [t.id for t in timeline2],
    [705362027901009921, 705358901349388289, 705357122859614209, 705354682886201348,
     705352657234481152, 705235874632638464, 705234524012224512, 705230846249324544,
     705229104426176514, 705227593205878784, 705213112463577088, 705179512254951424,
     705174403164860416, 705174362530430976, 705173204948656129, 705170546003218432,
     705149907183738880, 705148300064907265, 705119106052308992, 705102512718782465])

## Make predictions

- Use the SVM classifier to classify each tweet in timeline as a positive tweet or a negative tweet.

In [ ]:
def predict(clf, timeline):
    '''
    Uses a classifier ("clf") to classify each tweet in
    "timeline" as a positive tweet or a negative tweet.
    
    Parameters
    ----------
    clf: A Pipeline instance.
    timeline: A tweepy.models.ResultSet instance.
    
    Returns
    -------
    A numpy array.
    '''
    
    # YOUR CODE HERE
    
    return y_pred

In [ ]:
pred1 = predict(clf, timeline1)
print('{} has {} positive tweets and {} negative tweets.'.format(
    'Hillary Clinton', (pred1 == 1).sum(), (pred1 == 0).sum()))

In [ ]:
assert_is_instance(pred1, np.ndarray)
assert_array_equal(
     pred1,
     [1., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
      1., 1., 1., 1., 0., 1., 1., 1., 1., 0.]
    )

In [ ]:
pred2 = predict(clf, timeline2)
print('{} has {} positive tweets and {} negative tweets.'.format(
    'Donald Trump', (pred2 == 1).sum(), (pred2 == 0).sum()))

In [ ]:
assert_is_instance(pred2, np.ndarray)
assert_array_equal(
     pred2,
     [0., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
      1., 1., 0., 0., 1., 0., 0., 1., 1., 1.]
    )